In [14]:
import numpy as np
import matplotlib.pyplot as plt

In [91]:
def generate_timeline(spreading_factor, bandwidth, samples_per_chip, symbols_number = 1):
    '''
    This function generates a discrete timeline for a given spreading factor, bandwidth and samples per chip.
    The timeline is generated for one symbol time by default.
    '''
    # spreading_factor: spreading factor
    # bandwidth: bandwidth
    # samples_per_chip: samples per chip

    symbol_time = (2**spreading_factor) / bandwidth

    # Number of evenly-spaced time samples.
    # A plus one has to be added in order correctly generate the timeline
    num_time_samples= (2**spreading_factor) * samples_per_chip * symbols_number + 1

    timeline =  np.linspace(0, symbol_time * symbols_number, num_time_samples)

    # Remove the last element of the timeline due to the discretization

    timeline = timeline[:-1]

    return timeline


In [81]:
def generate_frequency_evolution(timeline, symbols, sf, bw):
    '''
    This function generates the linear frequency evolution for a given timeline, symbols, spreading factor and bandwidth.
    Parameters:
    timeline: discrete timeline
    symbols: array of symbols
    sf: spreading factor
    bw: bandwidth 
    '''
    # We only need a single symbol period to generate the frequency evolution
    timeline = timeline[:(len(timeline) // len(symbols))]

    symbol_duration = (2**sf) / bw
    frequency_slope = bw / symbol_duration
    frequency_evolution = np.array([])
    
    for symbol in symbols:
        
        y_intercept = symbol * ( bw / (2**sf))
        instantaneous_frequency = ( y_intercept + frequency_slope * (timeline) ) % bw
        frequency_evolution = np.concatenate((frequency_evolution, instantaneous_frequency))
        print(len(frequency_evolution))

    return frequency_evolution

In [108]:
def generate_signal(timeline, symbols, samples_per_chip, sf, bw):
    '''
    This function generates the complex signal for a given timeline, symbols, spreading factor, bandwidth and samples per chip.
    Parameters:
    timeline: discrete timeline
    symbols: array of symbols
    samples_per_chip: samples per chip
    sf: spreading factor
    bw: bandwidth
    '''
    # We only need a single symbol period to generate the signal
    timeline = timeline[:(len(timeline) // len(symbols))]

    symbol_duration = (2**sf) / bw
    integrated_frequency_slope = np.float64(bw / symbol_duration * 0.5)

    power_normalizer = 1 / np.sqrt(2**sf * samples_per_chip)

    signal = np.array([])

    for symbol in symbols:
        y_intercept = symbol * ( bw / (2**sf))
        instantaneous_frequency = ( y_intercept + integrated_frequency_slope * timeline ) % bw
        phase = 2 * np.pi * instantaneous_frequency * timeline
        signal_sample = power_normalizer * np.exp(1j * phase)
        signal = np.concatenate((signal, signal_sample))
    
    return signal

In [109]:
## Demodulation
# Generate downchirp
def generate_downchirp(sf, bw, timeline, also_return_freq=False, samples_per_chip = 1):
    '''
    This function generates a downchirp signal for a given spreading factor, bandwidth, timeline and samples per chip.
    Parameters:
    sf: spreading factor
    bw: bandwidth
    timeline: discrete timeline
    also_return_freq: boolean to also return the frequency evolution of the downchirp across the timeline
    samples_per_chip: samples per chip
    '''
    # sf: spreading factor
    # freq: frequency
    # timeline: timeline
    y_intercept = bw
    theoretical_symbol_time = (2**sf) / bw
    freqs = []
    integrated_freqs = []
    slope = -bw / theoretical_symbol_time
    coeff = 1/(np.sqrt(2**sf * samples_per_chip) )
    signal = []

    for i in range(len(timeline)):
        freq = y_intercept + slope * timeline[i]
        integrated_freq = y_intercept + 0.5 * slope * timeline[i]

        inst_phase = 2 * np.pi * (integrated_freq) * timeline[i]
        signal.append( coeff * np.exp(1j * inst_phase))
        freqs.append(freq)
        

    if also_return_freq:
        return signal, freqs
    return signal

def correlate(signal, sf, downchirp, return_symbol_instead=False):
    '''
    This function correlates a signal with a downchirp signal, resulting in the demodulation of the message.
    Parameters:
    signal: complex signal (array)
    sf: spreading factor
    downchirp: downchirp signal (array)
    return_symbol_instead: boolean to return the symbol instead of the fourier transform
    '''
    product = []
    for i in range(len(signal)):
        product.append(signal[i] * downchirp[i])

    fourier_transform = np.fft.fft(product)
    
    if return_symbol_instead:
        return np.argmax(fourier_transform) % 2**sf

    return fourier_transform


In [47]:
# # Testing the function "generate_timeline" for multi-symbol time (2 symbols)
# # make the generate_timeline function also return the erased sample timeline[-1] in order to test
# sf = 7
# bw = 125
# spc = 1
# symbols_number = 2
# time_test_1, er_s_1 = generate_timeline(sf, bw, spc, 1)
# print(len(time_test_1))
# time_test_2, er_s_2 = generate_timeline(sf, bw, spc, symbols_number)
# print(len(time_test_2))

# plt.figure(figsize=(20,8))
# plt.plot(time_test_1,'o')
# plt.plot(time_test_2[:len(time_test_1)],'o')
# if (time_test_1 == time_test_2[:len(time_test_1)]).all():
#     plt.title("Test1: The first part of the second timeline is equal to the first timeline")
# plt.show()

# time_test_2 = time_test_2[len(time_test_1):]
# time_test_2 = time_test_2 - er_s_1

# plt.figure(figsize=(20,8))
# plt.plot(time_test_1,'o')
# plt.plot(time_test_2,'o')
# if np.allclose(time_test_1, time_test_2, rtol=1e-5, atol=1e-8):
#     plt.title("Test2: The second part of the second timeline is equal to the first timeline (with the correct offset, taking into account rounding errors)")
# plt.show()

In [83]:
# # Testing the function "generate_frequency_evolution"
# sf = 7
# bw = 125e3
# spc = 1
# symbols = [0, 10, 30, 60, 90]

# timeline = generate_timeline(sf, bw, spc, len(symbols))
# timeline_2 = generate_timeline(sf, bw, spc, 1)
# print(len(timeline))
# frequency_evolution = generate_frequency_evolution(timeline, symbols, sf, bw)
# plt.plot(timeline, frequency_evolution)

In [96]:
# # Test1: generate_signal
# sf = 7
# bw = 125e3
# samples_per_chip = 10
# symbols = [0]

# # working thing to compare
# def compute_auxiliary_phase_factor(symbol, timeline, sf, bw):
#     # bw: bandwidth
#     theoretical_symbol_time = (2**sf) / bw
#     slope = bw / (theoretical_symbol_time)
#     y_intercept = symbol * (bw / 2**sf)
#     integrated_freq = []
#     for i in range(len(timeline)):
#         instant_freq = y_intercept + 0.5 * slope * timeline[i]
#         if instant_freq > bw:
#             instant_freq = instant_freq - bw
#         integrated_freq.append(instant_freq)
    
#     return integrated_freq

# def generate_signal_w(sf, freq, timeline, samples_per_chip = 1):
#     # sf: spreading factor
#     # freq: frequency
#     # timeline: timeline
#     coeff = 1/(np.sqrt(2**sf * samples_per_chip) )
#     signal = []

#     for i in range(len(timeline)):

#         inst_phase = 2 * np.pi * freq[i] * timeline[i] 
#         signal.append( coeff * np.exp(1j * inst_phase) )

#     return signal


# time_test_1 = generate_timeline(sf, bw, samples_per_chip, len(symbols))
# signal_1 = generate_signal(time_test_1, symbols, samples_per_chip, sf, bw)
# signal_2 = generate_signal_w(sf, compute_auxiliary_phase_factor(symbols[0], time_test_1, sf, bw), time_test_1, samples_per_chip)
# plt.figure(figsize = (20, 8))
# plt.plot(time_test_1, np.real(signal_1))
# plt.plot(time_test_1, np.real(signal_2))
# if np.allclose(signal_1, signal_2):
#     print('Test 1 passed')
# plt.show()